In [ ]:
#%pip install gensim

In [ ]:
#%pip install spacy

In [ ]:
#%pip install pyLDAvis 

In [46]:
#Imports
import pandas as pd
import pickle
import os
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
import pandas as pd
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt

import gensim
from gensim.models.ldamodel import LdaModel
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim import matutils

import spacy

import pyLDAvis
import pyLDAvis.gensim_models

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
#Import soptify + genius dataset with song lyrics
df_lyrics = pd.read_pickle("df_rap_7k.pickle")

In [3]:
df_lyrics.head()

,track_id,album_id,artist_id,duration_ms,flg_explicit,url,track_name,track_popularity,track_type,track_danceability,...,track_release_year,track_release_decade,artist_name,artist_genres,artist_followers,artist_popularity,lyrics,lyrics_clean,lyrics_clean_split,lyrics_wordlen
0,3SUiKUnZlKYliid0ZlCmUY,0uYanB3WWNU2lT9AYxCstH,3M1quhETLChtt9NHKVaajX,189405,True,https://api.spotify.com/v1/tracks/3SUiKUnZlKYl...,Detroit Flow,44,track,0.793,...,2020,20,Quin Nfn,"[austin hip hop, southern hip hop, vapor trap]",257151,61,"Detroit Flow Lyrics\n(Ayy, DJ Showoutt, fuck g...",detroit flow fuck hold really like sound beats...,"[detroit, flow, fuck, hold, really, like, soun...",244
1,2jdWfnwHSZLuBFzsA6axWt,0iNKw3iqIEy2dh7RUdKzjY,0nQlnRkAhiWnSjTmB4JkfK,273099,True,https://api.spotify.com/v1/tracks/2jdWfnwHSZLu...,Handz in da Air,8,track,0.826,...,2018,10,Sons of Light,[],141,7,Pet Sound LyricsTurbid:\nYes it's Turbid and I...,pet sound yes turbid like reclaim freely like ...,"[pet, sound, yes, turbid, like, reclaim, freel...",209
2,2vcLUuRT3sqXnPJaIX5w5c,0VptfDjz26NkJS4b7bjmNK,0FtsMKmZEq8fBWqdSOWtqp,236619,True,https://api.spotify.com/v1/tracks/2vcLUuRT3sqX...,Substance (We Woke Up),20,track,0.731,...,2021,20,03 Greedo,"[cali rap, melodic rap, trap, vapor trap]",317202,71,"Substance (We Woke Up) Lyrics\nYeah, 03\nYeah,...",substance woke yeah yeah yeah yeah beat boy ta...,"[substance, woke, yeah, yeah, yeah, yeah, beat...",269
3,1r0faljjM2b876iNoaDUh5,0XAIjjN5qxViVS0Y5fYkar,0c173mlxpT3dSFRgMO8XPh,287320,True,https://api.spotify.com/v1/tracks/1r0faljjM2b8...,Sacrifices,56,track,0.779,...,2017,10,Big Sean,"[detroit hip hop, hip hop, pop, pop rap, rap, ...",9276030,84,"Sacrifices Lyrics\nBrrrt!\nI look up, so many ...",look many shit many young metro trust gon shoo...,"[look, many, shit, many, young, metro, trust, ...",421
4,3yhatBRYsMwuozCdQsGiTS,2t70lfxoIbKZHKFKN9ZnqE,0auu2itHTxEdAMRHvx7CyG,283200,False,https://api.spotify.com/v1/tracks/3yhatBRYsMwu...,Hard Core Rap Act,28,track,0.813,...,1995,90,Das EFX,"[east coast hip hop, gangster rap, hardcore hi...",301411,60,Hardcore Rap Act LyricsIntro/Hook (x2)\n(*The ...,hardcore rap act hook hardcore rap act back ha...,"[hardcore, rap, act, hook, hardcore, rap, act,...",267


In [4]:
#Create lemmatizer, stemmer and define stopwords

word_tokenizer=RegexpTokenizer(r'\w+')

wordnet_lemmatizer = WordNetLemmatizer()

stemmer = PorterStemmer()

stopwords_nltk = set(stopwords.words('english'))



In [5]:
#Save lyrics
temp = df_lyrics["lyrics_clean"].copy()

In [6]:
### Function to lemmatize tokenized documents (song lyrics here)
def lemmatizer(df):
    
    lemmed = []
    
    for song in df.lyrics_clean_split:
        lemmed.append([wordnet_lemmatizer.lemmatize(word) for word in song])
    return lemmed

In [296]:
### Fucntion to get dictionary and bag of words
#Returns dictionary and bow corpus
def get_dict_and_bow(data, extreme_list = None):
    
    #Create dictionary of words: mapping between word and id
    id2word = corpora.Dictionary(data)
    if extreme_list is not None:
        id2word.filter_extremes(no_below = extreme_list[0], no_above = extreme_list[1]) #Filter tokens that appear in less than and more than
    texts = data
    
    #Transform dictionary into bag of words
    corpus = [id2word.doc2bow(text) for text in texts]
    
    return id2word, corpus

In [19]:
### Function to get coherence score for particular model, dict and bow
#Returns the coherence model
def get_coherence(model, text, id_dict, method='c_v'):
    coherence_model = CoherenceModel(model=model, texts=text, dictionary=id_dict, coherence=method)
    coherence_lda = coherence_model.get_coherence()
    print(coherence_lda)
    return coherence_model

In [124]:
### Function to join document-topic scores back to document dataset
#Args: dataframe to join back into, corpus/bow, model to extract topics from 
def get_topics_in_df(df, corpus, model):

    #set minimum probability to 0 to ensure all topic scores are extracted
    get_document_topics = [model.get_document_topics(item,minimum_probability =0) for item in corpus]

    temp_df = []
    for i in range(len(get_document_topics)):
        temp_list = []
        for j in range(len(get_document_topics[i])):
            temp_tuple = get_document_topics[i][j]
            temp_list.append(temp_tuple[1])
        temp_df.append(temp_list)
    temp_df = pd.DataFrame(temp_df)
    df = df.join(temp_df)
    
    return df

In [9]:
lyrics_lemmatized_all = lemmatizer(df_lyrics)

In [131]:
#Create new column in df_lyrics with lemmed words
df_lyrics['lyrics_clean_split_lemmed'] = lyrics_lemmatized_all

In [295]:
id2word_all, corpus_all = get_dict_and_bow(lyrics_lemmatized_all, [200, 0.95])

In [272]:
#Fit LDA model
ntopics = 8
lda_model_all = LdaModel(corpus = corpus_all,
                    id2word = id2word_all,
                    num_topics = ntopics, 
                    random_state = 123,
                    passes=4,
                    alpha=0.01,
                    eta=0.01,
                    per_word_topics=True)

In [273]:
#Show the LDA topics: topic 0
lda_model_all.show_topic(topicid=0, topn=20)


[('nigga', 0.048973463),
 ('got', 0.031892452),
 ('like', 0.03175187),
 ('shit', 0.03075559),
 ('bitch', 0.028975658),
 ('fuck', 0.02635007),
 ('get', 0.025956139),
 ('back', 0.015310178),
 ('know', 0.0152687),
 ('hit', 0.010618327),
 ('gon', 0.010378368),
 ('money', 0.009442544),
 ('put', 0.008538144),
 ('as', 0.008126787),
 ('cause', 0.008063398),
 ('make', 0.0075593074),
 ('real', 0.0073576933),
 ('bout', 0.00725545),
 ('see', 0.006579687),
 ('say', 0.0062552476)]

In [274]:
#Get coherence score
coherence_model_all = get_coherence(lda_model_all, text = lyrics_lemmatized_all, id_dict=id2word_all)

0.4244097745908735


In [275]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis_all = pyLDAvis.gensim_models.prepare(lda_model_all, corpus_all, id2word_all)
vis_all

C:\Users\louis\anaconda3\envs\INSY669\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.067580  0.082882       1        1  19.211342
4     -0.058930 -0.021645       2        1  17.803064
3     -0.069040 -0.061194       3        1  15.903166
2     -0.072764  0.164767       4        1  11.994580
7     -0.077025 -0.021235       5        1  11.214077
6      0.330531  0.045233       6        1  10.230290
5      0.084577 -0.114481       7        1   9.437727
1     -0.069770 -0.074328       8        1   4.205753, topic_info=      Term          Freq         Total Category  logprob  loglift
400   feat  34706.000000  34706.000000  Default  30.0000  30.0000
173   yeah  27847.000000  27847.000000  Default  29.0000  29.0000
68     one  10099.000000  10099.000000  Default  28.0000  28.0000
64   nigga  14985.000000  14985.000000  Default  27.0000  27.0000
215   time  10209.000000  10209.000000  Default  26.0000  26.0000
..     ...           ...           ...      ...      ...      ...
51    like    662.854935  34321.477332   Topic8  -4.3892  -0.7783
55    make    440.538667   8276.825961   Topic8  -4.7978   0.2355
47    know    463.024131  18957.079470   Topic8  -4.7480  -0.5434
302    day    352.857094   4892.873734   Topic8  -5.0197   0.5392
157  right    332.812775   6077.175346   Topic8  -5.0782   0.2640

[478 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
874       1  0.039211    ace
874       2  0.060777    ace
874       3  0.001961    ace
874       4  0.021566    ace
874       5  0.001961    ace
...     ...       ...    ...
114       4  0.057218  young
114       5  0.049261  young
114       6  0.577084  young
114       7  0.068729  young
114       8  0.015405  young

[2498 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 4, 3, 8, 7, 6, 2])

In [276]:
#Name the topics
topic_names = {
    0:"Violence/Gangsta",
    1:"Musicality",
    2:"Struggle/Mortality/Spirituality",
    3:"Flex",
    4:"Romance/Sexuality",
    5:"Feats/Artists",
    6:"Black?",
    7:"Party"
}

In [277]:
#Join topics back to dataframe
df_lyrics_topics = df_lyrics.copy()
df_lyrics_topics = get_topics_in_df(df = df_lyrics, corpus = corpus_all, model = lda_model_all)

In [278]:
df_lyrics_topics.rename(columns = topic_names, inplace = True)

In [279]:
df_lyrics_topics

,track_id,album_id,artist_id,duration_ms,flg_explicit,url,track_name,track_popularity,track_type,track_danceability,...,lyrics_wordlen,lyrics_clean_split_lemmed,Violence/Gangsta,Musicality,Struggle/Mortality/Spirituality,Flex,Romance/Sexuality,Feats/Artists,Black?,Party
0,3SUiKUnZlKYliid0ZlCmUY,0uYanB3WWNU2lT9AYxCstH,3M1quhETLChtt9NHKVaajX,189405,True,https://api.spotify.com/v1/tracks/3SUiKUnZlKYl...,Detroit Flow,44,track,0.793,...,244,"[detroit, flow, fuck, hold, really, like, soun...",0.861611,0.026577,0.016043,0.000052,0.095562,0.000052,0.000052,0.000052
1,2jdWfnwHSZLuBFzsA6axWt,0iNKw3iqIEy2dh7RUdKzjY,0nQlnRkAhiWnSjTmB4JkfK,273099,True,https://api.spotify.com/v1/tracks/2jdWfnwHSZLu...,Handz in da Air,8,track,0.826,...,209,"[pet, sound, yes, turbid, like, reclaim, freel...",0.000083,0.029386,0.039138,0.000083,0.534294,0.247273,0.000083,0.149659
2,2vcLUuRT3sqXnPJaIX5w5c,0VptfDjz26NkJS4b7bjmNK,0FtsMKmZEq8fBWqdSOWtqp,236619,True,https://api.spotify.com/v1/tracks/2vcLUuRT3sqX...,Substance (We Woke Up),20,track,0.731,...,269,"[substance, woke, yeah, yeah, yeah, yeah, beat...",0.452609,0.010736,0.000047,0.000047,0.089101,0.000047,0.014476,0.432938
3,1r0faljjM2b876iNoaDUh5,0XAIjjN5qxViVS0Y5fYkar,0c173mlxpT3dSFRgMO8XPh,287320,True,https://api.spotify.com/v1/tracks/1r0faljjM2b8...,Sacrifices,56,track,0.779,...,421,"[look, many, shit, many, young, metro, trust, ...",0.145224,0.062657,0.331924,0.275086,0.000030,0.024432,0.000030,0.160618
4,3yhatBRYsMwuozCdQsGiTS,2t70lfxoIbKZHKFKN9ZnqE,0auu2itHTxEdAMRHvx7CyG,283200,False,https://api.spotify.com/v1/tracks/3yhatBRYsMwu...,Hard Core Rap Act,28,track,0.813,...,267,"[hardcore, rap, act, hook, hardcore, rap, act,...",0.240039,0.000050,0.000050,0.000050,0.759658,0.000050,0.000050,0.000050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6778,3YzxXVNHGZF09eGqhsYuHI,6CM5qhYBvpgYNek5kYwuOJ,20qISvAhX20dpIbOOzGK3q,187175,True,https://api.spotify.com/v1/tracks/3YzxXVNHGZF0...,The Pressure,56,track,0.573,...,192,"[pressure, pressure, pressure, feel, pressure,...",0.000087,0.000087,0.000087,0.550972,0.301167,0.147426,0.000087,0.000087
6779,7KIbDUwumrpG5f30kEYW1v,6Gdt5ogiuJ9knp8Q5148ea,1yxSLGMDHlW21z4YXirZDS,213066,False,https://api.spotify.com/v1/tracks/7KIbDUwumrpG...,Pump It,0,track,0.648,...,179,"[pump, pump, pump, pump, pump, pump, turn, rad...",0.000067,0.000067,0.046769,0.000067,0.778281,0.000067,0.000067,0.174617
6780,4a4dFaA4NfXU5A3snq2ZMa,4MloRVOQvX5YAfu2ik7OA9,3BeQqzKdlARoOd6y30kCO2,263773,True,https://api.spotify.com/v1/tracks/4a4dFaA4NfXU...,Made Man,12,track,0.647,...,184,"[made, man, singing, background, blessing, liv...",0.065969,0.000078,0.000078,0.354307,0.545522,0.033890,0.000078,0.000078
6781,2HIjQZsmscClqRGbfzwAgK,24jAbLMFzCTlIv3kQ94HwK,50co4Is1HCEo8bhOyUWKpn,210733,True,https://api.spotify.com/v1/tracks/2HIjQZsmscCl...,Diamonds (feat. Gunna),55,track,0.919,...,256,"[highest, lowest, young, stoner, life, stoner,...",0.000056,0.000056,0.009212,0.000056,0.000056,0.000056,0.990451,0.000056


In [221]:
#Export songs to pickle 
df_lyrics_topics.to_pickle("df_lyrics_topics.pickle")

# Topics for Nas Songs Only

In [280]:
df_nas = df_lyrics[df_lyrics.artist_name == "Nas"]

In [300]:
text_nas = df_nas.lyrics_clean_split_lemmed.tolist()

In [301]:
#
id2word_nas, corpus_nas = get_dict_and_bow(text_nas)

In [331]:
ntopics = 8
lda_model_nas = LdaModel(corpus = corpus_nas,
                    id2word = id2word_nas,
                    num_topics = ntopics, 
                    random_state = 123,
                    passes=8,
                    alpha = 0.01,
                    eta = 0.01,
                    per_word_topics=True)

In [332]:
#Get coherence score
coherence_model_nas = get_coherence(lda_model_nas, text = text_nas, id_dict=id2word_nas)

0.2715007411314377


In [333]:
# Visualize the topics
vis_nas = pyLDAvis.gensim_models.prepare(lda_model_nas, corpus_nas, id2word_nas)
vis_nas

C:\Users\louis\anaconda3\envs\INSY669\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.306247  0.021048       1        1  26.443707
7      0.093947 -0.014273       2        1  25.037045
0      0.103839 -0.007793       3        1  14.925801
3      0.090985 -0.005824       4        1  12.147116
5      0.077600 -0.028432       5        1  11.639368
4      0.033336 -0.141681       6        1   4.936185
1      0.089596  0.177691       7        1   4.619406
6     -0.183056 -0.000736       8        1   0.251371, topic_info=           Term         Freq        Total Category  logprob  loglift
935        feat  3067.000000  3067.000000  Default  30.0000  30.0000
153        love   380.000000   380.000000  Default  29.0000  29.0000
216       world   375.000000   375.000000  Default  28.0000  28.0000
150        like   702.000000   702.000000  Default  27.0000  27.0000
167         one   449.000000   449.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
957         run     0.499061   101.749322   Topic8  -5.6468   0.6685
68         time     0.542090   352.063745   Topic8  -5.5641  -0.4901
1523  winehouse     0.424288    28.590242   Topic8  -5.8091   1.7756
1506       ross     0.469519    90.578320   Topic8  -5.7078   0.7237
1962    highest     0.449307    60.115975   Topic8  -5.7518   1.0897

[665 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
3442      6  1.057883  acceptable
4623      7  0.950865       ackee
1467      1  0.969156      action
1467      5  0.020620      action
916       4  0.994452      advice
...     ...       ...         ...
1321      2  0.204320         yes
1321      4  0.286048         yes
1321      5  0.040864         yes
1321      6  0.122592         yes
1321      7  0.408639         yes

[1375 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 8, 1, 4, 6, 5, 2, 7])